In [1]:
%pip install scikit-learn
%pip install datasets
%pip install transformers
%pip install transformers[torch]
%pip install evaluate
%pip install pandas==2.0.3
%pip install torch
%pip install joblib

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated pa

In [2]:
import sys
sys.path.append('/content/drive/MyDrive/Faks/research_uiktp')

In [3]:
import joblib
import torch
import sklearn
import evaluate
import numpy as np
import pandas as pd
from data_utils import rename_columns
from datasets import DatasetDict, Dataset
from make_dataset import split_dataset      # Module not found on google collab
from transformers import BertTokenizer, BertModel, AutoModelForSequenceClassification, TrainingArguments, Trainer

In [4]:
CSV_DATASET_PATH = "./processed_data/processed_data.csv"
#CSV_DATASET_PATH = "/content/drive/MyDrive/Faks/research_uiktp/processed_data/processed_data.csv"   # directory not found on google collab
dataframe = pd.read_csv(CSV_DATASET_PATH)
dataframe = rename_columns(dataframe)
print(dataframe)

                                                    text  label
0      This really should get fixed in the JBoss.org ...      4
1      We can still mention it, but since it's not ne...      5
2      When using MassIndexer and FSDirectory all is ...      4
3               Should improve performance at query time     10
4      https://github.com/hibernate/hibernate-orm/pul...      1
...                                                  ...    ...
37176  When viewing the quickstart/archetype on a pho...      1
37177  See https://community.jboss.org/message/647192...      4
37178  Not sure if this is going to make it to 1.0, o...      1
37179  At the very least we need some basic informati...      1
37180  Something like :<!--        <p>Discover more a...      1

[37181 rows x 2 columns]


In [5]:
train_set, test_set, validation_set = split_dataset(dataframe, train_set_length=.8, test_set_length=.1, validation_set_length=.1, axis=0)
print(train_set)
print(test_set)
print(validation_set)

                                                    text  label
0      This really should get fixed in the JBoss.org ...      4
1      We can still mention it, but since it's not ne...      5
2      When using MassIndexer and FSDirectory all is ...      4
3               Should improve performance at query time     10
4      https://github.com/hibernate/hibernate-orm/pul...      1
...                                                  ...    ...
29739  This [query|https://issues.jboss.org/issues/?j...      3
29740  This [query|https://issues.jboss.org/issues/?j...      3
29741  In JBDS-2803, it was agreed that the wording o...      7
29742  MapContent uses a CopyOnWriteArrayList as its ...      3
29743  Replace the use of synchronized methods with R...      3

[29744 rows x 2 columns]
                                                    text  label
29744  After creating an openshift user using default...      6
29745  {{ComplexAttributeConverterFactory}} is suppos...      9
29746  Create 

In [6]:
dataset = DatasetDict(
    {
        "train":Dataset.from_dict(train_set.to_dict('list')),
        "test":Dataset.from_dict(test_set.to_dict('list')),
        "validation":Dataset.from_dict(validation_set.to_dict('list'))
    }
)
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 29744
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 3719
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 3718
    })
})


In [7]:
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

def tokenize_function(jira_tasks, column_name="text"):
    ret = tokenizer(jira_tasks[column_name], padding="max_length", truncation=True)
    return ret

tokenized_datasets = dataset.map(tokenize_function, batched=True)
train_set = tokenized_datasets["train"]
test_set = tokenized_datasets["test"]
validation_set = tokenized_datasets["validation"]
print(train_set)
print(test_set)
print(validation_set)

Map:   0%|          | 0/29744 [00:00<?, ? examples/s]

Map:   0%|          | 0/3719 [00:00<?, ? examples/s]

Map:   0%|          | 0/3718 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 29744
})
Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 3719
})
Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 3718
})


In [8]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [9]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=11)

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    print("GPU is not available. CPU will be used to train the model")
    device = torch.device("cpu")
model.to(device)

training_args = TrainingArguments(output_dir="training_logs", evaluation_strategy="epoch")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=test_set,
    compute_metrics=compute_metrics
)
trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPU is not available. CPU will be used to train the model


C:\Users\Teo\AppData\Roaming\Python\Python311\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/11154 [00:00<?, ?it/s]

{'loss': 2.2594, 'learning_rate': 4.775865160480545e-05, 'epoch': 0.13}
{'loss': 2.2477, 'learning_rate': 4.551730320961091e-05, 'epoch': 0.27}
{'loss': 2.2366, 'learning_rate': 4.3275954814416356e-05, 'epoch': 0.4}


In [ ]:
print("Training finished")
model_filename = "trained_classifier.sav"
joblib.dump(model, model_filename)

In [11]:
loaded_model = joblib.load(model_filename)
print("Model loaded")

RuntimeError: [enforce fail at ..\caffe2\serialize\inline_container.cc:337] . unexpected pos 381348032 vs 381347920

Training finished


OSError: [Errno 28] No space left on device

EOFError: 